<h1>Whatersed</h1>
<p>baseado no código apresentado pelo prof Msc. Aldísio Medeiros</p>
Utiliza as funções presente na biblioteca OpenCV e numpy

com algumas modificações minimas 
o arquivo watershed1 utiliza as funções do skimage

In [2]:
#bibliotecas

import numpy as np
import cv2
from matplotlib import pyplot as plt

from skimage import data



In [3]:
img = cv2.imread('originais/coins.jpg') #imagem baixada do scikit images

#img = data.coins() 

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

cv2.imshow('original',img)
cv2.imwrite('original_water.png',img)


True

# Generate an initial image with two overlapping circles
x, y = np.indices((80, 80))
x1, y1, x2, y2 = 28, 28, 44, 52
r1, r2 = 16, 20
mask_circle1 = (x - x1)**2 + (y - y1)**2 < r1**2
mask_circle2 = (x - x2)**2 + (y - y2)**2 < r2**2
img = np.logical_or(mask_circle1, mask_circle2)


In [ ]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
cv2.imshow('1',opening)
cv2.imwrite('watershed1.png',opening)


# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
cv2.imshow('2',sure_bg)
cv2.imwrite('watershed1.png',sure_bg)


# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

cv2.imshow('3',dist_transform)
cv2.imwrite('watershed3.png',dist_transform)

cv2.imshow('4',sure_fg)
cv2.imwrite('watershed4.png',sure_fg)


# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

cv2.imshow('5',unknown)

# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)


img[markers == -1] = [255,0,0]

cv2.imshow('7',img)
cv2.imwrite('watershed7.png',img)
cv2.waitKey()